In [1]:
!pip install -U spacy
!pip install spacy_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 40.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.8/49.8 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.0/45.0 kB 4.7 MB/s eta 0:00:00
  Attempting uninstall: spacy
    Found existing installation: spacy 3.6.1
    Uninstalling spacy-3.6.1:
      Successfully uninstalled spacy-3.6.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
en-core-web-sm 3.6.0 requires spacy<3.7.0,>=3.6.0, but you have spacy 3.7.2 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 197.8/197.8 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.0/314.0 kB 23.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 27.4 MB/s eta 0:00:00
     ━━━━━━━

In [2]:
import spacy
import spacy_transformers
from spacy.tokens import DocBin
from tqdm import tqdm
import json
import pandas as pd

In [3]:
spacy.__version__


'3.7.2'

In [ ]:
!nvidia-smi


Tue Oct 17 12:26:51 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P8     9W /  70W |      3MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!git clone https://github.com/laxmimerit/CV-Parsing-using-Spacy-3.git

Cloning into 'CV-Parsing-using-Spacy-3'...
remote: Enumerating objects: 82, done.
remote: Counting objects: 100% (76/76), done.
remote: Compressing objects: 100% (58/58), done.
remote: Total 82 (delta 14), reused 74 (delta 14), pack-reused 6
Receiving objects: 100% (82/82), 5.62 MiB | 17.22 MiB/s, done.
Resolving deltas: 100% (14/14), done.


In [ ]:
cv_data=json.load(open('/content/CV-Parsing-using-Spacy-3/data/training/train_data.json','r'))
# cv_data=json.load(open('/content/CV-Parsing-using-Spacy-3/data/training/Entity Recognition in Resumes.json','r'))

### Append New JSON Data to preexisting JSON data


In [ ]:
new_batch1=json.load(open('/content/1410202.json','r'))
# print(new_batch1[0])
import re

def trim_entity_spans(data: list) -> list:
    # Removes leading and trailing white spaces from entity spans.
    invalid_span_tokens = re.compile(r'\s')

    cleaned_data = []
    for text, annotations in data:
        entities = annotations['entities']
        valid_entities = []
        for start, end, label in entities:
            valid_start = start
            valid_end = end
            while valid_start < len(text) and invalid_span_tokens.match(
                    text[valid_start]):
                valid_start += 1
            while valid_end > 1 and invalid_span_tokens.match(
                    text[valid_end - 1]):
                valid_end -= 1
            valid_entities.append([valid_start, valid_end, label])
        cleaned_data.append([text, {'entities': valid_entities}])

    return cleaned_data

cleaned_data=trim_entity_spans(new_batch1)

In [ ]:
for item in cleaned_data:
  cv_data.append(item)

In [ ]:
len(cv_data)

259

# **Writing JSON Data to file**

In [ ]:
import json

# Data to be written
with open("/content/cv_data.json", "w") as outfile:
    json.dump(cv_data, outfile)

# **Loading JSON Data from file**

In [ ]:
import json

# Opening JSON file
with open('/content/drive/MyDrive/Colab Notebooks/cv_data1.json', 'r') as openfile:
    cv_data = json.load(openfile)

len(cv_data)

271

In [ ]:
!python -m spacy init fill-config /content/CV-Parsing-using-Spacy-3/data/training/base_config.cfg /content/CV-Parsing-using-Spacy-3/data/training/config.cfg

2023-10-15 09:16:36.304874: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
✔ Auto-filled config with all values
✔ Saved config
/content/CV-Parsing-using-Spacy-3/data/training/config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [ ]:
def get_spacy_doc(file,data):
  nlp=spacy.blank('en')
  db=DocBin()

  for text,annot in tqdm(data):
    doc=nlp.make_doc(text)
    annot=annot['entities']

    ents=[]
    entity_indices=[]

    for start,end,label in annot:
      skip_entity=False
      for idx in range(start,end):
        if idx in entity_indices:
          skip_entity=True
          break
      if skip_entity==True:
        continue

      entity_indices=entity_indices+list(range(start,end))

      try:
        span = doc.char_span(start,end,label=label,alignment_mode='strict')
      except:
        continue

      if span is None:
        err_data=str([start,end])+"  "+str(text)+'\n'
        file.write(err_data)
      else:
        ents.append(span)

    try:
      doc.ents=ents
      db.add(doc)
    except:
      pass

  return db

In [ ]:
from sklearn.model_selection import train_test_split
train,test=train_test_split(cv_data,test_size=0.2)

In [ ]:
len(train),len(test)

(216, 55)

In [ ]:
file=open('error.txt','w')
db=get_spacy_doc(file,train)
db.to_disk('train_data.spacy')
db=get_spacy_doc(file,test)
db.to_disk('test_data.spacy')
file.close()

100%|██████████| 55/55 [00:00<00:00, 69.68it/s]


### **Pretraining Section**

In [ ]:
!python -m spacy download en_core_web_lg

2023-10-16 14:10:43.455330: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 2.0 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In [ ]:
nlp=spacy.load("en_core_web_lg")
skill_pattern_path="/content/jz_skill_patterns.jsonl"

In [ ]:
ruler = nlp.add_pipe("entity_ruler")
ruler.from_disk(skill_pattern_path)
nlp.pipe_names

['tok2vec',
 'tagger',
 'parser',
 'attribute_ruler',
 'lemmatizer',
 'ner',
 'entity_ruler']

In [ ]:
!python -m spacy init fill-config /content/CV-Parsing-using-Spacy-3/data/training/base_config.cfg /content/CV-Parsing-using-Spacy-3/data/training/config_pretrain.cfg --pretraining


2023-10-14 09:06:44.346757: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
⚠ No tok2vec component found in the pipeline. If your tok2vec component
has a different name, you may need to adjust the tok2vec_model reference in the
[pretraining] block. If you don't have a tok2vec component, make sure to add it
to your [components] and the pipeline specified in the [nlp] block, so you can
pretrain weights for it.
✔ Auto-filled config with all values
✔ Saved config
/content/CV-Parsing-using-Spacy-3/data/training/config_pretrain.cfg
You can now add your data and train your pipeline:
python -m spacy train config_pretrain.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [ ]:
!python -m spacy pretrain /content/CV-Parsing-using-Spacy-3/data/training/config_pretrain.cfg ./output --paths.raw_text /content/pretrain_text.jsonl --gpu-id 0


2023-10-14 09:11:31.352316: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
⚠ Output directory is not empty.
It is better to use an empty directory or refer to a new output path, then the
new directory will be created for you.
ℹ Using GPU: 0
ℹ Loading config from:
/content/CV-Parsing-using-Spacy-3/data/training/config_pretrain.cfg
✔ Saved config file in the output directory

============== Pre-training tok2vec layer - starting at epoch 0 ==============
  #      # Words   Total Loss     Loss    w/s
  0        11206   44052.6162    44052   1084
  0        22739   86656.5293    42603   5329
  0        34011   125530.674    38874   5190
  0        45433   163244.424    37713   3728
  0        56536   198640.763    35396   4438
  0        67869   233568.830    34928   6967
  1        80647   272032.033    38463   7246
  1        91638   303985.483    31953   7531
  1       103329   338333.989    34348   7532
  1       114368   369577.171    31

## Training Section

In [ ]:
!python -m spacy train /content/config.cfg --output ./output --paths.train /content/train_data.spacy --paths.dev /content/test_data.spacy --gpu-id 0

2023-10-17 12:29:01.627502: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
ℹ Saving to output directory: output
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'transformer', 'ner']
ℹ Initial learn rate: 0.0
E    #       LOSS TOK2VEC  LOSS TRANS...  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  -------------  --------  ------  ------  ------  ------
  0       0          0.00        3836.42   1089.71    0.06    0.03    0.21    0.00
  2     200          0.00      

In [ ]:
nlp = spacy.load('/content/output/model-best')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


# New Section

In [29]:
!pip install PyMuPDF

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.6/30.6 MB 15.8 MB/s eta 0:00:00


In [30]:
import sys,fitz

In [31]:
def TextPreprocessor(fname):
  doc=fitz.open(fname)
  text=""
  for page in doc:
    text=text+str(page.get_text())
  ' '.join(text.split())
  return text

In [268]:
text=TextPreprocessor("/content/resume_data/Gunika_Dhingra3.pdf")

In [269]:
import re

# Define a regular expression pattern to match email addresses
email_pattern = r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,7}\b'

doc=nlp(text)
skills=[]
for ent in doc.ents:
  if ent.label_=="Skills" and ent.text not in skills:
    skills.append(ent.text)
  if ent.label_ == "Email Address":
    if re.search(email_pattern,ent.text):
      print(ent.text,"---->",ent.label_)
    else:
      print(ent.text,"---->","URL")
  else:
    print(ent.text,"---->",ent.label_)

Gunika Dhingra ----> Name
Full Stack ----> Designation
Web Developer ----> Designation
Faridabad ----> Location
gunikad22@gmail.com ----> Email Address
linkedin.com/in/gunika-dhingra
 ----> URL
github.com/Gunika2002
 ----> URL
Maharaja Surajmal Institute of Technology ----> College Name
Bachelor in Computer Science ----> Degree
Delhi ----> Location
Postman ----> Skills
RESTful ----> Skills
RESTful ----> Skills
ReactJS ----> Skills
NodeJS ----> Skills
NodeJS ----> Skills
ExpressJS ----> Skills
EJS ----> Skills
MongoDB ----> Skills
Mongoose ----> Skills
NodeJS ----> Skills
ExpressJS ----> Skills
EJS ----> Skills
MongoDB ----> Skills
Mongoose ----> Skills
database ----> Skills
C, C++ ----> Skills
HTML ----> Skills
CSS ----> Skills
Javascript ----> Skills
MongoDB ----> Skills
MySQL ----> Skills
VS Code ----> Skills
Codeblocks ----> Skills
Bootstrap ----> Skills
NodeJS ----> Skills
ExpressJS ----> Skills
EJS ----> Skills
Mongoose ----> Skills
ReactJS ----> Skills
Git ----> Skills
Github ---

In [270]:
skills

['Postman',
 'RESTful',
 'ReactJS',
 'NodeJS',
 'ExpressJS',
 'EJS',
 'MongoDB',
 'Mongoose',
 'database',
 'C, C++',
 'HTML',
 'CSS',
 'Javascript',
 'MySQL',
 'VS Code',
 'Codeblocks',
 'Bootstrap',
 'Git',
 'Github']

In [35]:
doc=nlp("I am Aryan Sethi. I work at Microsoft. I have 3 years of working experience. my email address is aryan.sethi18@gmail.com")
for ent in doc.ents:
  print(ent.text,"---->",ent.label_)

Aryan Sethi ----> Name
Microsoft ----> Companies worked at
aryan.sethi18@gmail.com ----> Email Address


In [ ]:
def nlpParser(text):
  pkl_filename="model.pkl"
  with open(pkl_filename,'rb') as f_in:
    model=pickle.load(f_in)

  doc=model(text)
  dict={}
  for ent in doc.ents:
    # print(ent.text,"---->",ent.label_)
    dict[ent.label_]=ent.text

  return dict

In [ ]:
data=nlpParser(text)
data

## **Building dataset for pretraining**

In [ ]:
new_cv_data=!git clone https://github.com/Sbhawal/resumeScraper

In [ ]:
#spacy
import spacy
from spacy.pipeline import EntityRuler
from spacy.lang.en import English
from spacy.tokens import Doc

#gensim
import gensim
from gensim import corpora

#Visualization
from spacy import displacy
from wordcloud import WordCloud
import plotly.express as px
import matplotlib.pyplot as plt

#Data loading/ Data manipulation
import pandas as pd
import numpy as np

#nltk
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download(['stopwords','wordnet','punkt'])
from nltk.tokenize import word_tokenize
#warning
import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
df = pd.read_csv("/content/resumeScraper/Resume.csv")
df = df.reindex(np.random.permutation(df.index))
data = df.copy().iloc[
    0:,
]
data.head()

,ID,Resume_str,Resume_html,Category
1766,19396040,INDUSTRIAL ENGINEERING INTERN ...,"<div class=""fontsize fontface vmargins hmargin...",ENGINEERING
727,36861863,ADVISOR Summary Customer Se...,"<div class=""fontsize fontface vmargins hmargin...",HEALTHCARE
1278,16893572,DIGITAL MARKETING MANAGER S...,"<div class=""fontsize fontface vmargins hmargin...",DIGITAL-MEDIA
860,27084878,OFFICE MANAGER Career Focus ...,"<div class=""fontsize fontface vmargins hmargin...",FITNESS
523,26456474,MDS COORDINATOR Skills ...,"<div class=""fontsize fontface vmargins hmargin...",ADVOCATE


In [ ]:
!pip install spacy_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 197.8/197.8 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 52.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.0/314.0 kB 33.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 31.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 63.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 57.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 26.6 MB/s eta 0:00:00


In [ ]:
!python -m spacy download en_core_web_lg

2023-10-17 12:28:29.133049: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 2.8 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In [ ]:
nlp = spacy.load("en_core_web_lg")
skill_pattern_path = "/content/jz_skill_patterns.jsonl"


In [ ]:
ruler = nlp.add_pipe("entity_ruler")
ruler.from_disk(skill_pattern_path)
nlp.pipe_names
# import pickle
# pkl_filename="/content/model(15_10_2023).pkl"

# with open(pkl_filename,'rb') as file:
#   pickle_model=pickle.load(file)

# nlp=pickle_model

['tok2vec',
 'tagger',
 'parser',
 'attribute_ruler',
 'lemmatizer',
 'ner',
 'entity_ruler']

In [ ]:
def get_skills(text):
    doc = nlp(text)
    myset = []
    subset = []
    for ent in doc.ents:
        if ent.label_ == "SKILL":
            subset.append(ent.text)
    myset.append(subset)
    return subset


def unique_skills(x):
    return list(set(x))

In [ ]:
clean = []
for i in range(data.shape[0]):
    review = re.sub(
        '(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?"',
        " ",
        data["Resume_str"].iloc[i],
    )
    review = review.lower()
    review = review.split()
    lm = WordNetLemmatizer()
    review = [
        lm.lemmatize(word)
        for word in review
        if not word in set(stopwords.words("english"))
    ]
    review = " ".join(review)
    clean.append(review)

In [ ]:
data["Clean_Resume"] = clean
# data["skills"] = data["Clean_Resume"].str.lower().apply(get_skills)
# data["skills"] = data["skills"].apply(unique_skills)
data.head()

,ID,Resume_str,Resume_html,Category,Clean_Resume
1766,19396040,INDUSTRIAL ENGINEERING INTERN ...,"<div class=""fontsize fontface vmargins hmargin...",ENGINEERING,industrial engineering intern summary seeking ...
727,36861863,ADVISOR Summary Customer Se...,"<div class=""fontsize fontface vmargins hmargin...",HEALTHCARE,advisor summary customer service representativ...
1278,16893572,DIGITAL MARKETING MANAGER S...,"<div class=""fontsize fontface vmargins hmargin...",DIGITAL-MEDIA,digital marketing manager summary creative man...
860,27084878,OFFICE MANAGER Career Focus ...,"<div class=""fontsize fontface vmargins hmargin...",FITNESS,office manager career focus exceptionally orga...
523,26456474,MDS COORDINATOR Skills ...,"<div class=""fontsize fontface vmargins hmargin...",ADVOCATE,md coordinator skill skill used enjoy challeng...


###[Utility] - Downloading more resumes

In [ ]:
resumes_text=[text for text in data['Resume_str']]

In [ ]:
import re

# Define a regular expression pattern to match email addresses
email_pattern = r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,7}\b'

index = 0

# Iterate through resumes_text
for text in resumes_text:
    # Check if the text contains an email address using the regular expression
    if re.search(email_pattern, text):
        index += 1
        if index == 20:
            break
        # Save the text to a file
        with open(f"/content/RASERDataset/{index}.txt", "w") as file1:
            file1.write(text)


## **[Utility] - Creating JSONL File**

In [ ]:
import json

text_data=[]

for text in resumes_text:
  text_data.append(text)

output_file_path = "/content/drive/MyDrive/Colab Notebooks/pretrain_text.jsonl"

with open(output_file_path, 'w', encoding='utf-8') as output_file:
    for text in text_data:
        json_object = {"text": text}
        output_file.write(json.dumps(json_object) + '\n')

print(f"JSONL file '{output_file_path}' created successfully.")

JSONL file '/content/drive/MyDrive/Colab Notebooks/pretrain_text.jsonl' created successfully.


## **[Utility] - Removing folder**

In [253]:
!rm -rf resume_data

### **Resume Scoring**

In [4]:
!pip install textract

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.9/106.9 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 15.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.0/69.0 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 55.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.5/471.5 kB 36.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.8/32.8 MB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 58.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.0/74.0 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.2/112.2 kB 11.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.5/128.5 kB 13.

In [5]:

from gensim.models import Word2Vec
import string
import re
import os
import collections
from os import listdir
from os.path import isfile, join
from joblib import dump, load
import pickle
from io import StringIO
import pandas as pd
from collections import Counter
import en_core_web_sm
nlp = en_core_web_sm.load()
from spacy.matcher import PhraseMatcher
from itertools import chain
import textract
from gensim.models import Word2Vec
import string
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import gensim
from gensim.models.phrases import Phraser, Phrases
import nltk
import collections
import re
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
nltk.download('punkt')
nltk.download('stopwords')

/usr/local/lib/python3.10/dist-packages/spacy/util.py:910: UserWarning: [W095] Model 'en_core_web_sm' (3.6.0) was trained with spaCy v3.6.0 and may not be 100% compatible with the current version (3.7.2). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [11]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [12]:
import pickle
with open("/content/drive/MyDrive/Colab Notebooks/model(15_10_2023).pkl","rb") as file:
  nlp=pickle.load(file)

In [13]:
#contains cleaned resume data
documents = []
with open('/content/drive/MyDrive/Colab Notebooks/resumes.txt', 'r') as file:
    for line in file:
        documents.append(str(line))

In [14]:
len(documents)

3446

In [15]:
import nltk
from nltk.corpus import stopwords
import string
def textPreprocessor(featureRecord):
    #Remove punctuations
    removePunctuation = [char for char in featureRecord if char not in string.punctuation]
    sentences =''.join(removePunctuation)

    #convert sentences to words
    words = sentences.split(" ")

    #normalize
    wordNormalized=[word.lower() for word in words]

    #remove stropwords
    finalWords=[word for word in wordNormalized if word not in stopwords.words("english")]

    return finalWords

In [16]:
textPreprocessor("Hi! I am Aryan Sethi. Welcome to India. Bachelor's")

['hi', 'aryan', 'sethi', 'welcome', 'india', 'bachelors']

In [17]:
#Create BOW
from sklearn.feature_extraction.text import CountVectorizer

#Initialize Count Vectorizer with our custom preprocessing function

wordVector = CountVectorizer(analyzer=textPreprocessor)

#Build Vocab
finalWordVocab = wordVector.fit(documents)

In [18]:
finalWordVocab.vocabulary_['javascript']

19191

In [63]:
vocabulary=finalWordVocab.vocabulary_
total_weight=sum(vocabulary.values())
vocabulary = {term: weight / total_weight * 1000 for term, weight in vocabulary.items()}
print(max(vocabulary.values()))
print(min(vocabulary.values()))

0.05178932104200114
0.0


In [64]:
import json

# Save vocabulary to a JSON file
with open('vocabulary.json', 'w') as f:
    json.dump(vocabulary, f)

# To load the vocabulary in the future
with open('vocabulary.json', 'r') as f:
    loaded_vocabulary = json.load(f)

In [65]:
vocabulary=loaded_vocabulary

In [66]:
def skill_finder(text):
  doc=nlp(text)
  doc=doc.ents
  skills=[]
  for ent in doc:
    label=ent.label_
    if label == "Skills" or label=="Designation" or label =="Degree":
      skills.append(ent.text)
  return list(set(skills))

In [67]:
import re
def cleanResume(fname):
    doc=fitz.open(fname)
    resumeText=""
    for page in doc:
      resumeText=resumeText+str(page.get_text())
    ' '.join(resumeText.split())
    resumeText = re.sub('http\S+\s*', ' ', resumeText)
    resumeText = re.sub('RT|cc', ' ', resumeText)
    resumeText = re.sub('#\S+', '', resumeText)
    resumeText = re.sub('@\S+', '  ', resumeText)
    resumeText = re.sub('[%s]' % re.escape("""!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~"""), ' ', resumeText)
    resumeText = re.sub(r'[^\x00-\x7f]',r' ', resumeText)
    resumeText = re.sub('\s+', ' ', resumeText)
    return resumeText

In [147]:
input_resume=cleanResume("/content/AryanSethiResumeAug2023.pdf")
input_jd=cleanResume("/content/[rtCamp]_Job_Application_Form_-_Campus_Joiners_2023-2024[1].pdf")

input_resume=skill_finder(input_resume)
input_jd=skill_finder(input_jd)

In [148]:
print(input_resume)
print(input_jd)

['NodeJs', 'A search', 'HTML', 'Nextjs', 'React Native', 'Breadth', 'Bull', 'ReactJS', 'Firebase', 'CSS', 'JavaScript', 'Nodejs', 'Scripting', 'authentication', 'MongoDB', 'Developer Intern', 'Redux', 'C and Python', 'Typescript', 'React js', 'Next js', 'Dijkstra', 'C', 'Reactjs', 'Bachelor of Technology Computer Science', 'Javascript', 'RESTful', 'PostgreSQL']
['Ph D Yes', 'WordPress', 'Web Engineer', 'Nginx', 'M Sc IT', 'PHP', 'HTML', 'CSS', 'JavaScript', 'MySQL', 'php', 'web engineer', 'Web Engineers', 'git']


In [149]:
improved_input_resume=[]
for i in range(len(input_resume)):
  items_in_text=textPreprocessor(input_resume[i])
  for item in items_in_text:
    improved_input_resume.append(item)

print(improved_input_resume)

['nodejs', 'search', 'html', 'nextjs', 'react', 'native', 'breadth', 'bull', 'reactjs', 'firebase', 'css', 'javascript', 'nodejs', 'scripting', 'authentication', 'mongodb', 'developer', 'intern', 'redux', 'c', 'python', 'typescript', 'react', 'js', 'next', 'js', 'dijkstra', 'c', 'reactjs', 'bachelor', 'technology', 'computer', 'science', 'javascript', 'restful', 'postgresql']


In [150]:
improved_input_jd=[]
for i in range(len(input_jd)):
  items_in_text=textPreprocessor(input_jd[i])
  for item in items_in_text:
    improved_input_jd.append(item)

print(improved_input_jd)

['ph', 'yes', 'wordpress', 'web', 'engineer', 'nginx', 'sc', 'php', 'html', 'css', 'javascript', 'mysql', 'php', 'web', 'engineer', 'web', 'engineers', 'git']


In [151]:
def Preprocessfile(filename):
  text = textract.process(filename)
  text= text.decode('utf-8').replace("\n", " ")
  # print(text)
  x=[]
  tokens=word_tokenize(text)
  tok=[w.lower() for w in tokens]
  table=str.maketrans('','',string.punctuation)
  strpp=[w.translate(table) for w in tok]
  words=[word for word in strpp if word.isalpha()]
  stop_words=set(stopwords.words('english'))
  words=[w for w in words if not w in stop_words]
  x.append(words)
  # print(x)
  res=" ".join(chain.from_iterable(x))
  return res

In [152]:
resume_skills=list(set(improved_input_resume))
jd_skills=list(set(improved_input_jd))

total_score=0
checked_skills=[]

for x in jd_skills:
  if x in resume_skills:
    checked_skills.append(x)
    if x in vocabulary:
      total_score += max(vocabulary[x]*10,2.5)
    else:
      total_score += 2.5

checked_skills=list(set(checked_skills))
for x in resume_skills:
  if x not in checked_skills:
    if x in vocabulary:
      total_score += max(vocabulary[x],0.25)
    else:
      total_score += 0.25

req_skills_len = len(jd_skills)
match = round(total_score / req_skills_len * 100, 1)
print(match)

101.8


Writing code to extract resumes from excel sheet

In [262]:
import pandas as pd

file_path = './resumes.xlsx'
df = pd.read_excel(file_path)

key_value_pairs_2d = []

for index, row in df.iterrows():
    first = row['First Name']
    second = row['Last Name']
    link = row['PDF Links']

    key_value_pair = [f"{first.replace(' ', '_')}_{second}", link]
    key_value_pairs_2d.append(key_value_pair)


pdf_links=key_value_pairs_2d

In [263]:
pdf_links

[['Aryan__Sethi',
  'https://drive.google.com/file/d/1chND4Om2GhNFZvDHszODwV1q6xn8EWSP/view'],
 ['Nayra_Sethi',
  'https://drive.google.com/file/d/1s_eH2OoRX3oL7NHKtgklTbS5QqyhjI2q/view?usp=sharing'],
 ['Gunika_Dhingra',
  'https://drive.google.com/file/d/1MysDxbbpEhsFJ0guKeNMj9siPsivHgTE/view?usp=sharing'],
 ['Tushar_Jha',
  'https://drive.google.com/file/d/1r1xaCidguB46x_HtM6rM45b-AsKhZp40/view?usp=sharing']]

In [ ]:
!pip install gdown

In [256]:
import gdown

def downloadResumes(name,google_drive_url,file_no):
  file_id = google_drive_url.split('/')[-2]

  output_path = f"/content/resume_data/{name}{file_no}.pdf"

  gdown.download(f"https://drive.google.com/uc?id={file_id}", output_path, quiet=False)

  print(f"Downloaded {output_path}")

In [264]:
for i in range(len(pdf_links)):
  downloadResumes(pdf_links[i][0],pdf_links[i][1],i+1)

Downloading...
From: https://drive.google.com/uc?id=1chND4Om2GhNFZvDHszODwV1q6xn8EWSP
To: /content/resume_data/Aryan__Sethi1.pdf
100%|██████████| 122k/122k [00:00<00:00, 68.7MB/s]


Downloaded /content/resume_data/Aryan__Sethi1.pdf


Downloading...
From: https://drive.google.com/uc?id=1s_eH2OoRX3oL7NHKtgklTbS5QqyhjI2q
To: /content/resume_data/Nayra_Sethi2.pdf
100%|██████████| 36.2k/36.2k [00:00<00:00, 15.4MB/s]


Downloaded /content/resume_data/Nayra_Sethi2.pdf


Downloading...
From: https://drive.google.com/uc?id=1MysDxbbpEhsFJ0guKeNMj9siPsivHgTE
To: /content/resume_data/Gunika_Dhingra3.pdf
100%|██████████| 132k/132k [00:00<00:00, 51.3MB/s]


Downloaded /content/resume_data/Gunika_Dhingra3.pdf


Downloading...
From: https://drive.google.com/uc?id=1r1xaCidguB46x_HtM6rM45b-AsKhZp40
To: /content/resume_data/Tushar_Jha4.pdf
100%|██████████| 132k/132k [00:00<00:00, 75.3MB/s]

Downloaded /content/resume_data/Tushar_Jha4.pdf


In [258]:
import os

def Score_Resumes(folder_path,jd_path):
  input_jd=cleanResume(jd_path)
  input_jd=skill_finder(input_jd)

  improved_input_jd=[]
  for i in range(len(input_jd)):
    items_in_text=textPreprocessor(input_jd[i])
    for item in items_in_text:
      improved_input_jd.append(item)

  jd_skills=list(set(improved_input_jd))
  req_skills_len = len(jd_skills)


  score_list={}
  for item in os.listdir(folder_path):
    item_path = os.path.join(folder_path, item)
    if os.path.isdir(item_path):
            continue
    input_resume=cleanResume(item_path)
    input_resume=skill_finder(input_resume)

    improved_input_resume=[]

    for i in range(len(input_resume)):
      items_in_text=textPreprocessor(input_resume[i])
      for _item in items_in_text:
        improved_input_resume.append(_item)

    resume_skills=list(set(improved_input_resume))

    total_score=0
    checked_skills=[]

    for x in jd_skills:
      if x in resume_skills:
        checked_skills.append(x)
        if x in vocabulary:
          total_score += max(vocabulary[x]*10,2.5)
        else:
          total_score += 2.5

    checked_skills=list(set(checked_skills))
    for x in resume_skills:
      if x not in checked_skills:
        if x in vocabulary:
          total_score += max(vocabulary[x],0.25)
        else:
          total_score += 0.25

    match = round(total_score / req_skills_len * 100, 1)
    score_list[item]=match

  return score_list

In [265]:
res=Score_Resumes("/content/resume_data","/content/[rtCamp]_Job_Application_Form_-_Campus_Joiners_2023-2024[1].pdf")
sorted_score_list = dict(sorted(res.items(), key=lambda item: item[1], reverse=True))
print(sorted_score_list)

{'Tushar_Jha4.pdf': 117.9, 'Aryan__Sethi1.pdf': 101.8, 'Nayra_Sethi2.pdf': 100.0, 'Gunika_Dhingra3.pdf': 80.4}
